---
© 2025 KR-Labs. All rights reserved.  
KR-Labs™ is a trademark of Quipu Research Labs, LLC, a subsidiary of Sudiata Giddasira, Inc.

SPDX-License-Identifier: Apache-2.0
---

# Tutorial 2: Business Forecasting with Prophet

**Author:** KRL Model Zoo Team  
**Affiliation:** KR-Labs  
**Version:** v1.0  
**Date:** October 25, 2025  
**License:** Apache 2.0  
**Tier:** Open-Source (Tier 1-3)

---

## Tutorial Overview

This tutorial demonstrates business forecasting using **Prophet**, a powerful forecasting tool developed by Meta (Facebook) that excels at handling trends, seasonality, and holiday effects in business time series data.

**Models Covered:**
- Prophet (additive and multiplicative seasonality)
- Trend changepoint detection
- Holiday effects modeling
- Uncertainty intervals

**Dataset:** Synthetic employment data (monthly, 2015-2024)

### Learning Objectives

By the end of this tutorial, you will be able to:
1. Apply Prophet models to business time series data
2. Detect and interpret trend changepoints
3. Model seasonal patterns and holiday effects
4. Generate forecasts with uncertainty intervals

### Prerequisites

- Basic Python programming knowledge
- Understanding of time series concepts (trend, seasonality)
- Familiarity with business metrics (employment, revenue, sales)

**Estimated Time:** 35-45 minutes

---

## Business Applications

Prophet is particularly well-suited for:

- **Revenue Forecasting:** Predict monthly/quarterly revenue with seasonal patterns
- **Demand Planning:** Forecast product demand with holiday effects
- **Workforce Planning:** Project employment needs and hiring trends
- **Capacity Planning:** Anticipate resource requirements with growth trends

---

## Data Provenance

**Source:** Synthetic data generated for educational purposes  
**Characteristics:**
- Monthly frequency (120 observations)
- Growth trend (0.2% monthly)
- Seasonal hiring patterns
- Shock events (recession/boom)
- 7 industry sectors

**Real-world Equivalents:**
- U.S. Bureau of Labor Statistics (BLS) employment data
- Business revenue/sales data
- Industry-specific hiring trends

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Import KRL Model Zoo Prophet wrapper
from krl_models.econometric import ProphetModel
from krl_core import ModelMeta

# Set display options
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Load Data

Prophet expects data with columns 'ds' (date) and 'y' (target variable).

In [ ]:
# Load GDP data and prepare for Prophet
df = pd.read_csv('../data/gdp_sample.csv')
df['date'] = pd.to_datetime(df['date'])

# Prophet requires 'ds' and 'y' columns
prophet_data = df[['date', 'gdp']].copy()
prophet_data.columns = ['ds', 'y']

print(f"Dataset shape: {prophet_data.shape}")
print(f"Date range: {prophet_data['ds'].min()} to {prophet_data['ds'].max()}")
prophet_data.head()

In [ ]:
# Visualize the data
plt.figure(figsize=(14, 6))
plt.plot(prophet_data['ds'], prophet_data['y'], linewidth=2)
plt.title('Quarterly GDP - Raw Data', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('GDP (Billions)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Model Configuration

Prophet automatically detects:
- Trend changes (changepoints)
- Yearly seasonality
- Weekly seasonality
- Holiday effects (if provided)

Key parameters:
- `changepoint_prior_scale`: Flexibility of trend changes (higher = more flexible)
- `seasonality_prior_scale`: Strength of seasonality
- `seasonality_mode`: 'additive' or 'multiplicative'

In [ ]:
# Split data
train_size = int(len(prophet_data) * 0.8)
train_data = prophet_data[:train_size].copy()
test_data = prophet_data[train_size:].copy()

print(f"Training observations: {len(train_data)}")
print(f"Test observations: {len(test_data)}")

In [ ]:
# Configure Prophet model
prophet_params = {
    'time_col': 'ds',
    'value_col': 'y',
    'changepoint_prior_scale': 0.05,  # Moderate trend flexibility
    'seasonality_prior_scale': 10.0,   # Strong seasonality
    'seasonality_mode': 'additive',    # Additive seasonality
    'yearly_seasonality': True,
    'weekly_seasonality': False,       # Not applicable for quarterly data
    'daily_seasonality': False
}

meta = ModelMeta(
    name="GDP_Prophet",
    version="1.0",
    author="Tutorial",
    description="Prophet model for GDP forecasting"
)

# Fit model
prophet_model = ProphetModel(train_data, prophet_params, meta)
result = prophet_model.fit()

print("\nProphet model fitted successfully!")

## Make Predictions

In [ ]:
# Forecast future periods
forecast = prophet_model.predict(train_data, steps=len(test_data))

# Extract forecast components
forecast_df = forecast.payload['forecast']
print(f"Forecast columns: {forecast_df.columns.tolist()}")
print(f"\nForecast shape: {forecast_df.shape}")
forecast_df.head()

In [ ]:
# Visualize Prophet forecast
plt.figure(figsize=(14, 6))

# Plot training data
plt.plot(train_data['ds'], train_data['y'], label='Training Data', color='blue', linewidth=2)

# Plot test data (actual)
plt.plot(test_data['ds'], test_data['y'], label='Actual', color='green', linewidth=2)

# Plot forecast
plt.plot(forecast_df['ds'], forecast_df['yhat'], label='Prophet Forecast', color='red', linewidth=2, linestyle='--')

# Plot confidence intervals
plt.fill_between(forecast_df['ds'], 
                 forecast_df['yhat_lower'], 
                 forecast_df['yhat_upper'], 
                 alpha=0.2, color='red', label='95% Confidence Interval')

plt.title('Prophet GDP Forecast', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('GDP (Billions)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate forecast accuracy
actual_values = test_data['y'].values
forecast_values = forecast_df['yhat'].values[:len(test_data)]

mape = np.mean(np.abs((actual_values - forecast_values) / actual_values)) * 100
rmse = np.sqrt(np.mean((actual_values - forecast_values) ** 2))

print(f"Prophet Forecast Accuracy:")
print(f"  MAPE: {mape:.2f}%")
print(f"  RMSE: {rmse:.2f}")

## Decompose Components

Prophet decomposes the forecast into:
- Trend
- Seasonal components
- Holidays (if provided)

In [ ]:
# Plot components
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Trend
axes[0].plot(forecast_df['ds'], forecast_df['trend'], linewidth=2, color='darkblue')
axes[0].set_title('Trend Component', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Trend')
axes[0].grid(True, alpha=0.3)

# Yearly seasonality
if 'yearly' in forecast_df.columns:
    axes[1].plot(forecast_df['ds'], forecast_df['yearly'], linewidth=2, color='darkgreen')
    axes[1].set_title('Yearly Seasonality', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Seasonal Effect')
    axes[1].grid(True, alpha=0.3)
else:
    axes[1].text(0.5, 0.5, 'No yearly seasonality detected', 
                 transform=axes[1].transAxes, ha='center', va='center', fontsize=12)
    axes[1].set_title('Yearly Seasonality', fontsize=14, fontweight='bold')

# Full forecast
axes[2].plot(train_data['ds'], train_data['y'], label='Training', color='blue', linewidth=2)
axes[2].plot(forecast_df['ds'], forecast_df['yhat'], label='Forecast', color='red', linewidth=2, linestyle='--')
axes[2].set_title('Full Forecast', fontsize=14, fontweight='bold')
axes[2].set_ylabel('GDP')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Changepoint Detection

Prophet automatically detects trend changes. Let's visualize them.

In [ ]:
# Get changepoints from model
if 'changepoints' in result.payload:
    changepoints = result.payload['changepoints']
    print(f"Number of changepoints detected: {len(changepoints)}")
    print(f"\nChangepoint dates:")
    for cp in changepoints[:5]:  # Show first 5
        print(f"  {cp}")
    
    # Plot with changepoints
    plt.figure(figsize=(14, 6))
    plt.plot(train_data['ds'], train_data['y'], linewidth=2, label='Data')
    plt.plot(forecast_df['ds'], forecast_df['trend'], linewidth=2, color='red', linestyle='--', label='Trend')
    
    # Mark changepoints
    for cp in changepoints:
        plt.axvline(x=cp, color='gray', linestyle=':', alpha=0.5)
    
    plt.title('Trend with Detected Changepoints', fontsize=16, fontweight='bold')
    plt.xlabel('Date')
    plt.ylabel('GDP')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("Changepoint information not available in model output")

## Cross-Validation

Prophet includes built-in cross-validation for time series.

In [ ]:
# Note: Cross-validation requires the underlying Prophet model
# This is a simplified demonstration

print("Cross-validation allows you to:")
print("- Assess forecast accuracy at different horizons")
print("- Tune hyperparameters (changepoint_prior_scale, seasonality_prior_scale)")
print("- Evaluate model stability over time")
print("\nFor production use, implement proper time series CV with rolling windows")

## Hyperparameter Tuning

Let's compare models with different changepoint_prior_scale values.

In [ ]:
# Test different flexibility levels
scales = [0.001, 0.05, 0.5]
results = []

for scale in scales:
    params = prophet_params.copy()
    params['changepoint_prior_scale'] = scale
    
    model = ProphetModel(train_data, params, meta)
    model.fit()
    forecast = model.predict(train_data, steps=len(test_data))
    
    forecast_values = forecast.payload['forecast']['yhat'].values[:len(test_data)]
    mape = np.mean(np.abs((actual_values - forecast_values) / actual_values)) * 100
    
    results.append({'scale': scale, 'mape': mape})
    print(f"Changepoint scale {scale:.3f}: MAPE = {mape:.2f}%")

# Find best scale
best_result = min(results, key=lambda x: x['mape'])
print(f"\nBest changepoint_prior_scale: {best_result['scale']} (MAPE: {best_result['mape']:.2f}%)")

## Key Takeaways

1. **Prophet** is designed for business forecasting with built-in handling of:
   - Trend changes and saturation
   - Multiple seasonality patterns
   - Holiday effects
   - Missing data

2. **Advantages:**
   - Robust to outliers
   - Handles missing data well
   - Intuitive parameters
   - Automatic changepoint detection
   - Works well with limited historical data

3. **Best Used For:**
   - Business metrics (sales, revenue, traffic)
   - Data with strong seasonal patterns
   - Forecasts with known future events (holidays, promotions)
   - Non-expert users who need reliable forecasts

4. **Hyperparameters:**
   - `changepoint_prior_scale`: Controls trend flexibility (0.001 = rigid, 0.5 = flexible)
   - `seasonality_prior_scale`: Controls seasonal strength
   - `seasonality_mode`: 'additive' (default) or 'multiplicative'

## Next Steps

- Add custom seasonalities (e.g., quarterly for business cycles)
- Include holiday effects (national holidays, company events)
- Use multiplicative seasonality for percentage-based effects
- Implement proper cross-validation for production models
- Compare Prophet with SARIMA for your specific use case

## Export Results & Reproducibility

This section exports model results, visualizations, and metadata for reproducibility.

In [ ]:
from datetime import datetime
from pathlib import Path
import json

# Create output directory
output_dir = Path('../outputs') / f'prophet_forecasting_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Exporting results to: {output_dir}\n")

# 1. Export data
train_data.to_csv(output_dir / 'train_data.csv', index=False)
test_data.to_csv(output_dir / 'test_data.csv', index=False)
print(" Exported training and test data")

# 2. Export forecast results
forecast_df = pd.DataFrame({
    'date': test_data['date'],
    'actual_employment': test_data['total_employment'].values,
    'prophet_forecast': forecast_values
})
forecast_df.to_csv(output_dir / 'forecast_results.csv', index=False)
print(" Exported forecast results")

# 3. Export metadata
metadata = {
    "tutorial": "02_business_forecasting_prophet.ipynb",
    "version": "v1.0",
    "execution_date": datetime.now().isoformat(),
    "models": ["Prophet"],
    "dataset": {
        "name": "employment_sample.csv",
        "records": len(df),
        "train_size": len(train_data),
        "test_size": len(test_data)
    },
    "reproducibility": {
        "random_seed": 42,
        "python_version": "3.9+",
        "required_packages": ["krl-model-zoo", "pandas", "numpy", "matplotlib"]
    }
}

with open(output_dir / 'execution_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)
print(" Exported execution metadata")

print(f"\n{'='*60}")
print("EXPORT COMPLETE")
print(f"{'='*60}")
print(f"\nAll results saved to: {output_dir}")

## Responsible Use & Limitations

### Ethical Considerations

1. **Data Privacy:**
   - This analysis uses aggregated synthetic data
   - Real applications should use publicly available aggregate employment data
   - Avoid using models for individual hiring decisions without proper validation

2. **Bias & Fairness:**
   - Employment models reflect historical patterns which may encode past discrimination
   - Consider demographic and geographic equity when interpreting forecasts
   - Validate forecasts against diverse subpopulations

3. **Limitations:**
   - Synthetic data for demonstration purposes only
   - Real forecasts require domain expertise and validation
   - Prophet assumes smooth trends and may miss sudden structural breaks
   - Forecast accuracy decreases with prediction horizon
   - External shocks (pandemics, recessions, policy changes) not automatically captured

4. **Recommended Use Cases:**
   -  Educational purposes and learning
   -  Strategic planning and scenario analysis
   -  Budget forecasting and resource allocation
   -  Demand planning with uncertainty quantification
   -  High-stakes hiring/firing decisions without human review
   -  Regulatory compliance without expert validation
   -  Individual-level predictions or decisions

5. **Model Assumptions:**
   - Prophet assumes piecewise linear or logistic trends
   - Seasonality patterns assumed to be stable over time
   - Holiday effects require domain knowledge to specify
   - Model sensitive to outliers in recent data

### Best Practices

- Always validate forecasts against holdout test data
- Use multiple models for comparison and ensemble forecasting
- Document assumptions, limitations, and data sources
- Consult domain experts for business context and interpretation
- Update models regularly with new data (monthly/quarterly)
- Monitor forecast performance continuously and adjust as needed
- Consider multiple scenarios (optimistic, base, pessimistic)

For questions about responsible use: info@krlabs.dev

## References

1. **Taylor, S. J., & Letham, B.** (2018). Forecasting at scale. *The American Statistician*, 72(1), 37-45. https://doi.org/10.1080/00031305.2017.1380080

2. **Hyndman, R. J., & Athanasopoulos, G.** (2021). *Forecasting: Principles and Practice* (3rd ed.). OTexts. https://otexts.com/fpp3/

3. **Harvey, A. C.** (1990). *Forecasting, Structural Time Series Models and the Kalman Filter*. Cambridge University Press.

4. **Meta (Facebook) Prophet Documentation.** (2024). https://facebook.github.io/prophet/

5. **U.S. Bureau of Labor Statistics.** (2024). *Current Employment Statistics*. https://www.bls.gov/ces/

6. **Cleveland, R. B., Cleveland, W. S., McRae, J. E., & Terpenning, I.** (1990). STL: A seasonal-trend decomposition procedure based on loess. *Journal of Official Statistics*, 6(1), 3-73.

---

## Citation

To cite this tutorial:

```bibtex
@misc{krl_prophet_forecasting_2025,
  title = {Tutorial 2: Business Forecasting with Prophet},
  author = {KRL Model Zoo Team},
  year = {2025},
  publisher = {KR-Labs},
  url = {https://github.com/KR-Labs/krl-model-zoo},
  note = {Tutorial from KRL Model Zoo v1.0.0}
}
```

To cite KRL Model Zoo:

```bibtex
@software{krl_model_zoo_2025,
  title = {KRL Model Zoo: Production-Grade Models for Socioeconomic Analysis},
  author = {KR-Labs},
  year = {2025},
  url = {https://github.com/KR-Labs/krl-model-zoo},
  version = {1.0.0},
  license = {Apache-2.0}
}
```

---

<div style="text-align: center; padding: 20px 0; border-top: 2px solid #333; margin-top: 40px;">
  <p style="font-size: 14px; font-weight: bold; margin-bottom: 5px;">
    KR-Labs | Data-Driven Economic Analysis
  </p>
  <p style="font-size: 12px; color: #666; margin: 5px 0;">
    Contact: <a href="mailto:info@krlabs.dev">info@krlabs.dev</a>
  </p>
  <p style="font-size: 11px; color: #666; margin: 5px 0;">
    © 2025 KR-Labs. All rights reserved.<br>
    <strong>KR-Labs™</strong> is a trademark of Quipu Research Labs, LLC, a subsidiary of Sudiata Giddasira, Inc.
  </p>
  <p style="font-size: 11px; color: #666; margin: 5px 0;">
    <a href="https://www.apache.org/licenses/LICENSE-2.0" target="_blank">Apache 2.0 License</a> | 
    <a href="https://github.com/KR-Labs/krl-model-zoo" target="_blank">GitHub Repository</a>
  </p>
</div>